In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import numpy as np
import random
import os
from shutil import copyfile
import shutil
import matplotlib.pyplot as plt 

In [ ]:
print(len(os.listdir('/gdrive/My Drive/lunar/DataSet/data/Large')))
print(len(os.listdir('/gdrive/My Drive/lunar/DataSet/data/Small')))

In [ ]:
try:
  os.mkdir('/gdrive/My Drive/lunar/DataSet/train')
  os.mkdir('/gdrive/My Drive/lunar/DataSet/test')
  os.mkdir('/gdrive/My Drive/lunar/DataSet/train/large')
  os.mkdir('/gdrive/My Drive/lunar/DataSet/train/small')
  os.mkdir('/gdrive/My Drive/lunar/DataSet/test/large')
  os.mkdir('/gdrive/My Drive/lunar/DataSet/test/small')
except:
  pass

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):     
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]
    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

In [ ]:
source_large= '/gdrive/My Drive/lunar/DataSet/data/Large/'
source_small= '/gdrive/My Drive/lunar/DataSet/data/Small/'
dest_train_large= '/gdrive/My Drive/lunar/DataSet/train/large/'
dest_train_small= '/gdrive/My Drive/lunar/DataSet/train/small'
dest_test_large= '/gdrive/My Drive/lunar/DataSet/test/large/'
dest_test_small= '/gdrive/My Drive/lunar/DataSet/test/small'
split= 0.9

In [ ]:
#split_data(source_large, dest_train_large, dest_test_large, split)
#split_data(source_small, dest_train_small, dest_test_small, split)

In [ ]:
print(len(os.listdir(dest_train_large)))
print(len(os.listdir(dest_train_small)))
print(len(os.listdir(dest_test_large)))
print(len(os.listdir(dest_test_small)))

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

training_data = train_datagen.flow_from_directory('/gdrive/My Drive/lunar/DataSet/train/', batch_size=16, class_mode='binary', target_size=(720,480))
testing_data = test_datagen.flow_from_directory('/gdrive/My Drive/lunar/DataSet/test/', batch_size=16, class_mode='binary', target_size=(720,480))

In [ ]:
base_vgg19= tf.keras.applications.VGG19(include_top=False, weights=None, input_shape=(720,480,3))
base_vgg19.trainable=False

In [ ]:
base_vgg19.summary()

In [ ]:
features = base_vgg19.output

In [ ]:
features.shape

In [ ]:
x= tf.keras.layers.Flatten()(features)
x= tf.keras.layers.Dense(16, activation='relu')(x)
x= tf.keras.layers.Dense(16, activation='relu')(x)
x= tf.keras.layers.Dense(16, activation='relu')(x)
x= tf.keras.layers.Dense(1, activation='sigmoid')(x) 

In [ ]:
model_vgg19= tf.keras.Model(base_vgg19.input, x)

In [ ]:
model_vgg19.summary()

In [ ]:
model_vgg19.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), 
                    metrics=['accuracy'], 
                    loss='binary_crossentropy')

In [ ]:
history = model_vgg19.fit(training_data, 
                          epochs=10, 
                          steps_per_epoch=10, 
                          validation_data=testing_data)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()